Copyright 2019 DoiT International LTD <br>

Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br>
You may obtain a copy of the License at<br>

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.<br>

In [1]:
query_extract_features = """WITH daily_agg AS (

    SELECT
        
        export_day,
        SUM(clean_filled_daily_cost) OVER(PARTITION BY TIMESTAMP_TRUNC(export_day, MONTH) ORDER BY export_day) daily_clean_filled_cost_sum_from_month_start,
        AVG(clean_filled_daily_cost) OVER(ORDER BY UNIX_SECONDS(export_day) RANGE BETWEEN 15552000 PRECEDING AND CURRENT ROW) avg_last_180_days,   
        STDDEV(clean_filled_daily_cost) OVER(ORDER BY UNIX_SECONDS(export_day) RANGE BETWEEN 15552000 PRECEDING AND CURRENT ROW) std_last_180_days, 
        days_left   
        FROM(
        SELECT
          *, 
          EXTRACT(DAY FROM DATETIME_SUB(DATETIME_ADD(DATETIME_TRUNC(DATETIME(export_day), MONTH), INTERVAL 1 MONTH), INTERVAL 1 DAY)) - EXTRACT(DAY FROM export_day) days_left,
          IF(clean_daily_cost IS NULL, AVG(clean_daily_cost) OVER(ORDER BY UNIX_SECONDS(export_day) RANGE BETWEEN 345600 PRECEDING AND 86400 PRECEDING ), clean_daily_cost) clean_filled_daily_cost

FROM(
    SELECT *,
    IF(((total_exact_daily < lower_bound) OR (total_exact_daily > upper_bound)), NULL, total_exact_daily) AS clean_daily_cost
FROM(

    SELECT *,
        PERCENTILE_CONT(total_exact_daily, .025) OVER() AS lower_bound,
        PERCENTILE_CONT(total_exact_daily, .975) OVER() AS upper_bound
    FROM(
    SELECT
  
        TIMESTAMP_TRUNC(export_time, DAY) export_day,
        (SUM(CAST(cost * 1000000 AS int64))
            + SUM(IFNULL((SELECT SUM(CAST(c.amount * 1000000 as int64))
            FROM UNNEST(credits) c), 0))) / 1000000 AS total_exact_daily
                        
    FROM `{billing_export_table}`

    WHERE (billing_account_id in ('XXXXXX-XXXXXX-XXXXXX', 'XXXXXX-XXXXXX-XXXXXX') AND cost > 0
        
    GROUP BY 1))))
),

customer_invoice AS (

SELECT
  TIMESTAMP_TRUNC(export_time, MONTH) export_month,
  (SUM(CAST(cost * 1000000 AS int64))
    + SUM(IFNULL((SELECT SUM(CAST(c.amount * 1000000 as int64))
                  FROM UNNEST(credits) c), 0))) / 1000000
    AS total_exact_monthly
FROM `{billing_export_table}`

WHERE billing_account_id in ('XXXXXX-XXXXXX-XXXXXX', 'XXXXXX-XXXXXX-XXXXXX')
GROUP BY 1
)
SELECT *
FROM daily_agg
LEFT JOIN customer_invoice ON DATETIME(customer_invoice.export_month) = DATETIME_ADD(DATETIME_TRUNC(DATETIME(daily_agg.export_day), MONTH), INTERVAL 3 MONTH) """

In [ ]:
next_month_train_model = """

CREATE OR REPLACE MODEL 
  `cost_pred.linear_model_next_month_pred` 
OPTIONS
  (model_type='linear_reg',
    ls_init_learn_rate=.015, 
    l1_reg=0.1,
    l2_reg=0.1,
    data_split_method='seq',
    data_split_col='split_col',
    max_iterations=30 
    ) AS
SELECT
    total_exact_monthly label,
    export_month split_col,
    
    daily_clean_filled_cost_sum_from_month_start,
    avg_last_30_days,
    days_left

FROM
  `cost_pred.next_month_pred_data`  
  WHERE export_month >= '2018-01-01' AND export_month < '2019-06-01'

"""

next_month_prediction = """

SELECT
  export_month,
  export_day,
  ROUND(predicted_label) predicted,
  total_exact_monthly actual_month_cost,
  ROUND(100 * (ABS(predicted_label - total_exact_monthly) /  total_exact_monthly),1) abs_err
FROM
  ML.PREDICT(MODEL `cost_pred.linear_model_next_month_pred`,
    (
    SELECT
      export_month,
      export_day,
    
    daily_clean_filled_cost_sum_from_month_start,
    avg_last_30_days,
    days_left,
    
    ROUND(total_exact_monthly) total_exact_monthly
    FROM
      `cost_pred.next_month_pred_data`
    WHERE export_month >= '2019-06-01' AND export_month < '2019-10-01'))
ORDER BY
  export_month


"""

In [ ]:
next_3_months_train_model = """

CREATE OR REPLACE MODEL 
  `cost_pred.linear_model_3_months_pred`
OPTIONS
  (model_type='linear_reg', 
    ls_init_learn_rate=.015, 
    l1_reg=0.1,
    l2_reg=0.1,
    data_split_method='seq',
    data_split_col='split_col',
    max_iterations=30 
    ) AS
SELECT
    total_exact_monthly label, 
    export_month split_col,
    -- independent variables:
    daily_clean_filled_cost_sum_from_month_start,
    (avg_last_180_days + std_last_180_days) as avg_std_180,
    days_left


FROM
  `cost_pred.3_month_pred_data`  
  WHERE export_month >= '2019-01-01' AND export_month < '2019-08-01' 

"""

next_3_months_prediction = """

SELECT
  export_month,
  export_day,
  ROUND(predicted_label) predicted,
  total_exact_monthly actual_month_cost,
  ROUND(100 * (ABS(predicted_label - total_exact_monthly) /  total_exact_monthly),1) abs_err
FROM
  ML.PREDICT(MODEL `cost_pred.linear_model_3_months_pred`,
    (
    SELECT
      
      export_month,
      export_day,
    
    daily_clean_filled_cost_sum_from_month_start,
    (avg_last_180_days + std_last_180_days) as avg_std_180,
    days_left,
    ROUND(total_exact_monthly) total_exact_monthly
    FROM
      `cost_pred.3_month_pred_data`
    WHERE export_month >= '2019-08-01' AND export_month < '2019-10-01'))
ORDER BY
  export_month


"""